In [1]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 592, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 592 (delta 125), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (592/592), 194.79 KiB | 8.47 MiB/s, done.
Resolving deltas: 100% (299/299), done.
Installing RAPIDS remaining 25.04 libraries
Using Python 3.11.12 environment at: /usr
Resolved 173 packages in 1.44s
 Downloaded cuproj-cu12
 Downloaded rmm-cu12
 Downloaded shapely
 Downloaded libkvikio-cu12
 Downloaded datashader
 Downloaded dask
 Downloaded pylibcugraph-cu12
 Downloaded ucx-py-cu12
 Downloaded cudf-cu12
 Downloaded bokeh
 Downloaded cugraph-cu12
 Downloaded cuspatial-cu12
 Downloaded librmm-cu12
 Downloaded cucim-cu12
 Downloaded cuml-cu12
 Downloaded libcuspatial-cu12
 Downloaded libraft-cu12
 Downloaded pylibcudf-cu12
 Downloaded raft-dask-cu12
 Downloaded libcudf-cu12
 Downloaded libcuml-cu12
 Downloaded libcuvs

In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 19.7 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
# import cudf
import optuna

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from cuml.svm import SVC
from cuml.linear_model import LogisticRegression


from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [4]:
nltk.data.path.append('/usr/lib/nltk_data')
nltk.download('stopwords', download_dir='/usr/lib/nltk_data')
nltk.download('punkt_tab', download_dir='/usr/lib/nltk_data')

[nltk_data] Downloading package stopwords to /usr/lib/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /usr/lib/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
df = pd.read_excel("drive/MyDrive/IMDB_Dataset.xlsx")
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [6]:
stop_words = set(stopwords.words('english'))

In [7]:
def preprocessing_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(filtered_tokens)

In [8]:
df['review'] = df['review'].apply(preprocessing_text)
df['sentiment'] = LabelEncoder().fit_transform(df['sentiment'])

In [9]:
df

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,1
1,wonderful little production br br filming tech...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake thinks zombie...,0
4,petter mattei love time money visually stunnin...,1
...,...,...
49995,thought movie right good job creative original...,1
49996,bad plot bad dialogue bad acting idiotic direc...,0
49997,catholic taught parochial elementary schools n...,0
49998,going disagree previous comment side maltin on...,0


In [10]:
x = df['review']
y = df['sentiment']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

In [11]:
vectorizer = TfidfVectorizer(max_features=3000, sublinear_tf=True)

x_train_vectorized = vectorizer.fit_transform(x_train).toarray()
x_test_vectorized = vectorizer.transform(x_test).toarray()

In [ ]:
# x_train_dense = x_train_vectorized.toarray()
# x_test_dense = x_test_vectorized.toarray()


In [12]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_vectorized)
x_test_scaled = scaler.transform(x_test_vectorized)

In [ ]:
def objective(trial):
    C = trial.suggest_categorical('C', [0.1, 1, 10])

    model = SVC(C=C, random_state=42)

    score = cross_val_score(model, x_train_vectorized, y_train, scoring='accuracy', cv=3)

    return score.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

best_param_svc = study.best_params
print(f"best param svc: {best_param_svc}")

[I 2025-05-02 21:07:32,521] A new study created in memory with name: no-name-02bf859c-32a3-4404-874e-ea2ec2ef864f
[I 2025-05-02 21:09:19,903] Trial 0 finished with value: 0.8824749211050661 and parameters: {'C': 1}. Best is trial 0 with value: 0.8824749211050661.
[I 2025-05-02 21:11:00,081] Trial 1 finished with value: 0.8824749211050661 and parameters: {'C': 1}. Best is trial 0 with value: 0.8824749211050661.
[I 2025-05-02 21:13:40,543] Trial 2 finished with value: 0.8824249198550348 and parameters: {'C': 10}. Best is trial 0 with value: 0.8824749211050661.
[I 2025-05-02 21:15:23,474] Trial 3 finished with value: 0.8824749211050661 and parameters: {'C': 1}. Best is trial 0 with value: 0.8824749211050661.
[I 2025-05-02 21:17:58,744] Trial 4 finished with value: 0.8824249198550348 and parameters: {'C': 10}. Best is trial 0 with value: 0.8824749211050661.
[I 2025-05-02 21:20:34,163] Trial 5 finished with value: 0.8824249198550348 and parameters: {'C': 10}. Best is trial 0 with value: 0.8

best param svc: {'C': 1}


In [ ]:
def objective(trial):
    n_estimators = trial.suggest_categorical("n_estimators",[50, 100, 150, 200])
    max_depth = trial.suggest_int("max_depth", 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        criterion=criterion,
        random_state=42
    )

    score = cross_val_score(model, x_train_vectorized, y_train, scoring='accuracy', cv=5)

    return score.mean()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=15)

best_param_rf = study.best_params
print(f"best param rf: {best_param_rf}")

[I 2025-04-30 15:45:48,339] A new study created in memory with name: no-name-789ff2e7-e75b-4fcb-90c9-bf2c0063cc42
[I 2025-04-30 15:48:08,135] Trial 0 finished with value: 0.8299 and parameters: {'n_estimators': 200, 'max_depth': 18, 'min_samples_split': 7, 'criterion': 'gini'}. Best is trial 0 with value: 0.8299.
[I 2025-04-30 15:51:24,187] Trial 1 finished with value: 0.8356 and parameters: {'n_estimators': 150, 'max_depth': 27, 'min_samples_split': 9, 'criterion': 'gini'}. Best is trial 1 with value: 0.8356.
[I 2025-04-30 15:55:24,333] Trial 2 finished with value: 0.8383750000000001 and parameters: {'n_estimators': 100, 'max_depth': 42, 'min_samples_split': 6, 'criterion': 'gini'}. Best is trial 2 with value: 0.8383750000000001.
[I 2025-04-30 15:59:42,984] Trial 3 finished with value: 0.841325 and parameters: {'n_estimators': 100, 'max_depth': 48, 'min_samples_split': 11, 'criterion': 'gini'}. Best is trial 3 with value: 0.841325.
[I 2025-04-30 15:59:59,280] Trial 4 finished with val

best param rf: {'n_estimators': 200, 'max_depth': 43, 'min_samples_split': 6, 'criterion': 'entropy'}


In [ ]:
def objective(trial):
    C = trial.suggest_categorical("C", [0.1, 1, 10])

    model = LogisticRegression(
        C=C,
        penalty='l2',
        random_state=42
    )

    score = cross_val_score(model, x_train_vectorized, y_train, scoring='accuracy', cv=5)

    return score.mean()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=15)

best_param_lr = study.best_params
print(f"best param lr: {best_param_lr}")

[I 2025-04-30 16:41:24,306] A new study created in memory with name: no-name-e6ddd152-e557-406d-9f85-71633637e8de


[2025-04-30 16:41:24.308] [CUML] [info] Unused keyword parameter: random_state during cuML estimator initialization


[I 2025-04-30 16:41:28,055] Trial 0 finished with value: 0.86915 and parameters: {'C': 0.1}. Best is trial 0 with value: 0.86915.


[2025-04-30 16:41:28.060] [CUML] [info] Unused keyword parameter: random_state during cuML estimator initialization


[I 2025-04-30 16:41:28,394] Trial 1 finished with value: 0.86915 and parameters: {'C': 0.1}. Best is trial 0 with value: 0.86915.


...

[2025-04-30 16:41:33.151] [CUML] [info] Unused keyword parameter: random_state during cuML estimator initialization


[I 2025-04-30 16:41:33,524] Trial 13 finished with value: 0.8822749999999999 and parameters: {'C': 1}. Best is trial 2 with value: 0.8823000000000001.


[2025-04-30 16:41:33.527] [CUML] [info] Unused keyword parameter: random_state during cuML estimator initialization


[I 2025-04-30 16:41:33,904] Trial 14 finished with value: 0.8822000000000001 and parameters: {'C': 1}. Best is trial 2 with value: 0.8823000000000001.


best param lr: {'C': 1}


In [1]:
def objective(trial):
    hidden_layer_sizes = trial.suggest_categorical("hidden_layer_sizes", [50, 100, 150, 200])
    activation = trial.suggest_categorical("activation", ["relu", "tanh", "logistic"])
    solver = trial.suggest_categorical("solver", ["adam", "sgd"])
    alpha = trial.suggest_loguniform("alpha", 1e-5, 1e-2)

    model = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        alpha=alpha,
        early_stopping=True,
        random_state=42
    )

    score = cross_val_score(model, x_train_scaled, y_train, scoring="accuracy", cv=5)

    return score.mean()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

best_param_mlp = study.best_params
print(f"best param mlp: {best_param_mlp}")